# Créer une carte du niveau des eaux souterraines à partir des données de forage et d'un DEM.

## 1. Introduction
Dans cet exercice, vous apprendrez à convertir un fichier Comma Separated Values contenant les profondeurs et les coordonnées d'un forage en une carte PCRaster. Ensuite, nous allons interpoler les profondeurs de forage en un raster continu en utilisant la méthode de pondération de distance inverse IDW (Inverse Distance Weighting) et la méthode de Thiessen.

Importons d'abord PCRaster et définissons le chemin vers le dossier `Data` où nous avons les données d'entrée pour cet exercice.

In [ ]:
from pcraster import *
import os
print(os.getcwd())
os.chdir("./Data")

## 2. Convertir un fichier CSV au format de carte PCRaster
Ouvrez le fichier `boreholesdhxy.csv` dans le dossier `Data` dans un éditeur de texte et vérifiez le contenu.
Quelles colonnes sont les coordonnées et quelle colonne donne la profondeur du forage ?

PCRaster possède quelques applications en ligne de commande qui sont utiles pour ces tâches. Vous pouvez trouver ces applications [Ici](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/applications.html). Nous allons utiliser <code>col2map</code> pour convertir le fichier CSV en une carte PCRaster.

La syntaxe de <code>col2map</code> est <code>col2map [options] columnfile PCRresult</code>.
Ici, nous allons utiliser quelques options pour indiquer le type de données de sortie (Scalar) et l'ordre des x, y et de la colonne de données (v). Nous devons également indiquer une carte clone qui possède les propriétés matricielles (système de coordonnées, taille des cellules, nombre de lignes/colonnes) de la sortie souhaitée.

Nous pouvons exécuter des commandes en ligne de commande depuis Python en utilisant <code>os.system(stringWithCommand)</code> :

In [ ]:
command = "col2map -S -x 2 -y 3 -v 1 boreholesdxy.csv boreholedepth.map --clone clone.map"
os.system(command)

Lire le fichier `boreholedepth.map` dans la variable `BoreholeDepth`.

Visualiser `BoreholeDepth` avec Aguila. Parce que nous exécutons ce Jupyter Notebook localement, nous pouvons utiliser l'opération python `aguila`. La carte apparaîtra en arrière-plan, donc vous devrez peut-être la chercher.

In [ ]:
aguila(BoreholeDepth)

## 3. Interpoler la profondeur du forage avec IDW
Nous pouvons maintenant interpoler la profondeur du trou de forage au niveau des pixels. Nous allons d'abord appliquer la méthode de pondération de la distance inverse. Cette méthode a été implémentée sous la forme de l'opération [inversedistance operation](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_inversedistance.html) dans PCRaster.

La syntaxe est la suivante : 
```Python
Résultat = inversedistance( mask, points, idp, radius, maxNr)
```

Où 
* <code>mask</code> est un masque de l'étendue qui sera interpolée. Dans notre cas, nous utiliserons la carte de clonage. 
* <code>points</code> est la couche matricielle scalaire avec les pixels que vous voulez interpoler. Dans notre cas <code>BoreholeDepth</code>.
* <code>idp</code> est l'exposant de la fonction puissance pour les poids. Une valeur <code>idp</code> de 2 est couramment utilisée.
* <code>radius</code> indique la distance d'une cellule à l'intérieur de laquelle il faut prendre en compte les points d'interpolation. S'il est fixé à 0, il n'y a pas de limitation.
* <code>maxNr</code> limite le nombre de points utilisés pour l'interpolation. S'il est fixé à 0, il n'y a pas de maximum.

Lisons d'abord la carte de clone que nous utiliserons comme masque.

In [ ]:
Mask = readmap("clone.map")

Maintenant, nous interpolons les profondeurs des forages avec IDW en utilisant la ligne suivante :

In [ ]:
BoreholeDepthIDW = inversedistance( Mask, BoreholeDepth, 2, 0, 0)

Visualisons le résultat avec Aguila.

## 4. Interpoler la profondeur du forage avec les polygones de Thiessen
Une autre méthode d'interpolation consiste à utiliser les polygones dits de Thiessen.

Dans PCRaster, cela implique trois étapes :
1. Attribuer un ID unique à chaque forage
2. Attribuer à chaque pixel l'ID du trou de forage le plus proche.
3. Affecter la profondeur du forage aux ID de forage correspondants pour toutes les cellules.

Commençons par attribuer un ID unique à chaque trou de forage en utilisant l'opération <code>uniqueid</code>. L'entrée doit être un raster booléen.

Pour créer un raster booléen des forages, nous utilisons l'opération <code>defined</code>. Cela donnera le booléen True (vrai) pour toutes les cellules ayant une valeur et le booléen False (faux) pour toutes les cellules nodata.

In [ ]:
Boreholes = defined(BoreholeDepth)

Vérifiez le résultat avec Aguila.

Maintenant, nous attribuons un ID unique à chaque cellule booléenne True (vrai). Notez que nous voulons que la sortie soit nominale au lieu d'être scalaire. Par conséquent, nous enveloppons le <code>uniqueid</code> dans l'opération <code>nominal</code>.

In [ ]:
BoreholeID = nominal(uniqueid(Boreholes))

Vérifiez le résultat avec Aguila.

Affectons à chaque cellule son identifiant de forage proche. Nous utilisons l'opération <code>spreadzone</code>, dont la syntaxe est la suivante :

<code>Résultat = spreadzone(points, initialfrictiondist, friction)</code>.

Où :

* <code>points</code> est le raster avec les points à interpoler. Dans notre cas, BoreholeID.
* <code>initialfrictiondist</code> est la distance de friction initiale. Nous n'en avons pas besoin, donc nous la mettons à 0.
* <code>friction</code> est la friction pour chaque cellule, que nous gardons constante à 1.

In [ ]:
ThiessenID = spreadzone(BoreholeID,0,1)

Vérifiez le résultat avec Aguila.

L'étape finale consiste à affecter les valeurs de profondeur des eaux souterraines des forages aux polygones de Thiessen.
Nous pouvons le faire en calculant la profondeur minimale ou maximale par classe, ce qui permettra juste d'extrapoler la valeur à l'ensemble de la classe. Ici, nous utiliserons le <code>areamaximum</code> :

In [ ]:
BoreholeDepthThiessen = areamaximum(BoreholeDepth,ThiessenID)

Vérifiez le résultat et comparez-le avec l'interpolation IDW utilisant Aguila.

In [ ]:
aguila(BoreholeDepthIDW,BoreholeDepthThiessen)

Pouvez-vous écrire un script qui calcule les polygones de Thiessen ? Faites une fonction qui peut être appliquée à des données similaires à celles du raster `BoreholeDepth`.

*Par __[Hans van der Kwast](http://www.linkedin.com/in/jvdkwast)__*<br>
*__[IHE Delft Institute for Water Education](http://www.un-ihe.org)__*<br>
*Twitter: @hansakwast*

*Ces matériels sont en libre accès, sous la license [CC By-NC 4.0](https://creativecommons.org/licenses/by-nc/4.0/)*